# Card Launch Success
### JPMorgan SQL Interview Question

#### Question
Your team at JPMorgan Chase is soon launching a new credit card. You are asked to estimate how many cards you'll issue in the first month.

Before you can answer this question, you want to first get some perspective on how well new credit card launches typically do in their first month.

Write a query that outputs the name of the credit card, and how many cards were issued in its launch month. The launch month is the earliest record in the `monthly_cards_issued` table for a given card. Order the results starting from the biggest issued amount.

#### monthly_cards_issued Table:
| Column Name     | Type  |
|-----------------|-------|
| issue_month     | integer |
| issue_year      | integer |
| card_name       | string  |
| issued_amount   | integer |

#### Example Input:

| issue_month | issue_year | card_name              | issued_amount |
|-------------|------------|------------------------|---------------|
| 1           | 2021       | Chase Sapphire Reserve | 170000        |
| 2           | 2021       | Chase Sapphire Reserve | 175000        |
| 3           | 2021       | Chase Sapphire Reserve | 180000        |
| 3           | 2021       | Chase Freedom Flex     | 65000         |
| 4           | 2021       | Chase Freedom Flex     | 70000         |

#### Example Output:

| card_name              | issued_amount |
|------------------------|---------------|
| Chase Sapphire Reserve | 170000        |
| Chase Freedom Flex     | 65000         |

#### Explanation:
Chase Sapphire Reserve card was launched in January 2021 with an issued amount of 170,000 cards and the Chase Freedom Flex card was launched in March 2021 with an issued amount of 65,000 cards.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Initialize Spark session
spark = SparkSession.builder.master('local[1]').appName("CardLaunchSuccess").getOrCreate()

# Create data for the table
data = [
    (1, 2021, 'Chase Sapphire Reserve', 170000),
    (2, 2021, 'Chase Sapphire Reserve', 175000),
    (3, 2021, 'Chase Sapphire Reserve', 180000),
    (3, 2021, 'Chase Freedom Flex', 65000),
    (4, 2021, 'Chase Freedom Flex', 70000)
]

# Define schema for the DataFrame
columns = ['issue_month', 'issue_year', 'card_name', 'issued_amount']

# Create the DataFrame
df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()


+-----------+----------+--------------------+-------------+
|issue_month|issue_year|           card_name|issued_amount|
+-----------+----------+--------------------+-------------+
|          1|      2021|Chase Sapphire Re...|       170000|
|          2|      2021|Chase Sapphire Re...|       175000|
|          3|      2021|Chase Sapphire Re...|       180000|
|          3|      2021|  Chase Freedom Flex|        65000|
|          4|      2021|  Chase Freedom Flex|        70000|
+-----------+----------+--------------------+-------------+



In [2]:
from pyspark.sql.window import Window
winspec= Window.partitionBy('card_name').orderBy('issue_year','issue_month')

df\
    .withColumn('rnk', row_number().over(winspec))\
    .where(col('rnk')==1)\
    .select("card_name","issued_amount")\
    .orderBy('issued_amount',ascending=0).show()

+--------------------+-------------+
|           card_name|issued_amount|
+--------------------+-------------+
|Chase Sapphire Re...|       170000|
|  Chase Freedom Flex|        65000|
+--------------------+-------------+



In [3]:
df.createOrReplaceTempView('monthly_cards_issued')



In [4]:
%%sparksql
with cte as 
(   SELECT *, row_number()
          over(PARTITION BY card_name ORDER BY issue_year,issue_month) as rnk
    FROM monthly_cards_issued)

SELECT
      card_name,issued_amount
FROM cte
WHERE rnk=1
ORDER by 1 DESC

+--------------------+-------------+
|           card_name|issued_amount|
+--------------------+-------------+
|Chase Sapphire Re...|       170000|
|  Chase Freedom Flex|        65000|
+--------------------+-------------+

